In [1]:
data_var = '2024-01-31'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11322,2024-01-31,Eua Nba,00:00,Golden State Warriors,Philadelphia 76ers,1.63,2.36,233.5,1.88,2.04,-3.5,1.97,1.97,G6RriPB2,0.613497,0.423729,0.531915,0.490196,0.037226,0.8,0.6,NaN,NaN,307.300,199.077529,0.647828,173.448,68.049972,0.392336,220.012,182.380,259.20,133.12,0.0,0.0,0.0,0.0,0.258741,0.057723,0.000000,-2.32,-0.464,-1.357759,0.617497,0.4,-0.217497,-2.24,-0.448,-3.035714,0.583631,0.4,-0.183631
11323,2024-01-31,Eua Nba,21:00,Washington Wizards,Los Angeles Clippers,6.69,1.13,237.5,1.88,2.05,11.5,2.06,1.86,jPJelNeL,0.149477,0.884956,0.531915,0.487805,0.034433,0.2,0.4,NaN,NaN,457.276,216.100434,0.472582,213.292,97.042498,0.454975,464.092,218.388,371.52,192.24,1.0,0.0,0.0,0.0,1.005502,0.061175,0.072154,-5.00,-1.000,-5.690000,0.302311,0.2,-0.102311,-0.32,-0.064,-2.031250,0.549824,0.5,-0.049824
11324,2024-01-31,Eua Nba,21:30,Miami Heat,Sacramento Kings,1.93,1.93,228.5,1.89,2.03,1.5,1.81,2.10,ne5Rr1Jr,0.518135,0.518135,0.529101,0.492611,0.036269,0.2,0.4,NaN,NaN,230.858,115.288792,0.499393,200.406,72.819600,0.363360,194.664,254.810,170.10,133.90,0.0,0.0,0.0,0.0,0.000000,0.050508,0.104891,-3.63,-0.726,-1.280992,0.583990,0.4,-0.183990,-1.92,-0.384,-2.421875,0.581448,0.6,0.018552
11325,2024-01-31,Eua Nba,22:00,Houston Rockets,New Orleans Pelicans,2.27,1.68,227.5,1.80,2.08,2.5,1.97,1.97,EHyOsLYl,0.440529,0.595238,0.555556,0.480769,0.035767,0.4,0.4,NaN,NaN,271.954,134.134753,0.493226,226.534,115.856153,0.511429,285.394,307.700,159.82,427.84,0.0,0.0,0.0,0.0,0.211237,0.102057,0.000000,-0.47,-0.094,-13.510638,0.559239,0.5,-0.059239,-0.67,-0.134,-5.074627,0.508829,0.7,0.191171
11326,2024-01-31,Eua Nba,22:00,Oklahoma City Thunder,Denver Nuggets,1.66,2.04,228.5,1.82,1.99,-4.5,2.08,1.70,0YwzuJI7,0.602410,0.490196,0.549451,0.502513,0.092606,0.2,0.4,NaN,NaN,164.050,42.401416,0.258466,163.868,25.976388,0.158520,173.602,195.776,173.72,149.52,0.0,0.0,0.0,0.0,0.145244,0.063101,0.142170,0.64,0.128,5.156250,0.686425,0.8,0.113575,0.44,0.088,11.818182,0.629447,0.7,0.070553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11435,2024-01-31,República Tcheca Copa Da República Tcheca,14:00,Decin,Usti n. Labem,1.61,2.22,164.5,1.84,1.86,-4.5,2.02,1.71,ziMrg6Ai,0.621118,0.450450,0.543478,0.537634,0.071568,0.0,0.0,NaN,NaN,NaN,NaN,NaN,136.614,32.684654,0.239248,0.000,0.000,0.00,97.44,0.0,0.0,0.0,0.0,0.225240,0.007644,0.117535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11436,2024-01-31,Rússia Superliga,09:00,Barnaul,MBA Moscow 2,1.39,2.84,154.5,1.86,1.84,-7.5,2.06,1.65,pARA6XOi,0.719424,0.352113,0.537634,0.543478,0.071537,0.0,0.4,NaN,NaN,129.526,18.261563,0.140988,197.726,77.191575,0.390397,0.000,218.388,105.84,223.88,0.0,0.0,0.0,0.0,0.484778,0.007644,0.156288,0.00,0.000,inf,0.000000,0.0,0.000000,2.00,0.400,4.600000,0.000000,0.0,0.000000
11437,2024-01-31,Rússia Superliga,12:00,Zenit Petersburg 2,Dynamo Vladivostok,3.77,1.24,151.5,1.86,1.84,7.5,2.01,1.68,SYGF5ivb,0.265252,0.806452,0.537634,0.543478,0.071704,0.4,0.4,NaN,NaN,174.812,77.337689,0.442405,145.956,64.801944,0.443983,158.756,144.220,165.48,136.00,0.0,0.0,0.0,0.0,0.714164,0.007644,0.126474,-1.60,-0.320,-8.656250,0.000000,0.0,0.000000,-0.26,-0.052,-4.615385,0.000000,0.0,0.000000
11438,2024-01-31,Suécia Superettan,15:00,KFUM Fryshuset,Huddinge,1

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Philadelphia 76ers,233.5,1.88,1.0000,Over
1,21:00,Eua Nba,Washington Wizards,Los Angeles Clippers,237.5,1.88,1.0000,Over
2,21:30,Eua Nba,Miami Heat,Sacramento Kings,228.5,1.89,1.0000,Over
3,22:00,Eua Nba,Houston Rockets,New Orleans Pelicans,227.5,1.80,1.0000,Over
4,22:00,Eua Nba,San Antonio Spurs,Orlando Magic,226.5,1.82,0.9999,Over
5,22:30,Eua Nba,Brooklyn Nets,Phoenix Suns,231.5,1.90,1.0000,Over
6,15:00,Europa Euroliga,Monaco,Partizan,164.5,1.85,1.0000,Over
7,15:00,Europa Euroliga,Zalgiris Kaunas,Anadolu Efes,164.5,1.80,0.9750,Over
